In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
plt.rcParams["figure.figsize"] = (8, 6)
plt.rcParams["figure.dpi"] = 100
plt.style.use("bmh")

In [ ]:
os.chdir("..")

In [ ]:
df = pd.read_csv("data/precleaned.csv", index_col=0)

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df.head()

# Quickstart
1. [Import danych](#Import-danych-i-podstawowe-informacje-o-zbiorze)
2. [Wykresy](#Wykresy-i-wnioski-przyblizajace-zbior-danych)
3. [Potencjalne pytania](#Pytania,-ktore-moga-potencjalnie-naprowadzic-na-cechy,-ktore-maja-wieksze-znaczenie-dla-predykcji-targetu)
4. [Key takeaways](#Key-takeaways)

# Detekcja wartosci odstajacych

In [ ]:
df.describe()

## Cechy numeryczne

In [ ]:
subset = df.select_dtypes(exclude="object")

In [ ]:
fig, axs = plt.subplots(len(subset.columns), dpi=200, figsize=(12, 22))

for ind, feature in enumerate(subset.columns):
    sns.boxplot(data=df, x=df[feature], y="target", ax=axs[ind], orient="h")

plt.subplots_adjust(hspace=0.7)

### Spostrzeżenia
- Głównym targetem kampanii są osoby w wieku między 30 a 50 rokiem życia
- Osoby, które nie były chętne skorzystać z oferty szybciej kończyły rozmowę
- Przeważnie klienci zgadzali się przystać na ofertę po maksymalnie kilku rozmowach, prowadzenei większej ilości rozmów nie spowodowało, że chętniej się zgadzali na ofertę
- Na ofertę częściej zgadzali się klienci z mniejsza liczą pracowników( około 5100)

### wiek

In [ ]:
wiek_subset = np.abs(stats.zscore(df["wiek"]))
wiek_subset.plot(kind="hist", bins=6, logy=True, title="Zscore dla wieku");

In [ ]:
threshold = 3
df = df.iloc[np.where(wiek_subset <= threshold)[0]]

### dl_polaczenia

In [ ]:
sns.histplot(data=df.dl_polaczenia)

### Spostrzeżenia
- Ustandaryzowano wiek klientów
- Sporo osób praktycznie natychmiastowo, bądź po usłyszeniu, że jest to oferta kończy rozmowę. Jest to częste zachowanie osób, które nie akceptują żadnych ofert proponowanych przez rozmowy telefoniczne. Na takie zachowanie nie ma wplywu żaden czynnik taki jak wykształcenie, zawód czy stan cywilny.

In [ ]:
df[(df.dl_polaczenia <= 0)]

#### Nie nalezy porzucac tych obserwacji, gdyz sa naturalnym wynikiem rozlaczenia sie klienta z konsultantem

In [ ]:
df = df[~(df.dl_polaczenia <= 0)]

In [ ]:
subset = stats.boxcox(df["dl_polaczenia"])[0]

#### Sprawdzamy, czy _dl\_polaczenia_, po transformacji ma rozklad normalny.

In [ ]:
sns.histplot(data=subset);

In [ ]:
df[df.dl_polaczenia > 2000].target.value_counts()

#### Dlugie rozmowy maja duze znaczenie dla modelu, bo mozna zauwazyc, ze im dluzsze sa rozmowy, tym wieksza szansa na to, ze osoba bedzie zainteresowana depozytem, wiec nie sa to tzw. wartosci odstajace.

#### Kolumna _dl\_polaczenia_ jest jedynie cecha pozwalajaca na walidacje modelu, wiec zostanie ona pominieta.

### liczba_polaczen_aktualnej_kampanii

In [ ]:
pd.crosstab(index=df.liczba_polaczen_aktualnej_kampanii, columns=df.target).T

#### Na podstawie powyzszego zestawienia moznaby pokusic sie o porzucenie obserwacji, gdzie _liczba\_polaczen\_aktualnej\_kampanii_ przekracza wartosc 11, bo znaczaca wiekszosc takich obserwacji, sprowadza sie do nieudanej proby namowienia klienta do depozytu.

In [ ]:
df = df[df.liczba_polaczen_aktualnej_kampanii <= 18]

In [ ]:
sns.countplot(
    data=df[df.liczba_polaczen_aktualnej_kampanii < 20],
    x="liczba_polaczen_aktualnej_kampanii",
    hue="target",
);

### liczba_dni_od_ost_kontaktu

In [ ]:
sns.countplot(data=df, x="liczba_dni_od_ost_kontaktu", hue="target")
plt.yscale("log")

#### Jako ze jest to skala logarytmiczna, moze sie wydawac ze roznica miedzy iloscia ludzi, ktorzy zdecydowali sie na depozyt, a tymi ktorzy sie na to nie zdecydowali, jest minimalna, jednak w rzeczywistosci jest ona bardzo duza.

In [ ]:
df[df.liczba_dni_od_ost_kontaktu == 999].target.value_counts()

### WAZNE! Zastanowic sie co zrobic z tymi 999, czy zamienic je na pd.NA? Tutaj 999 oznacza, ze po prostu nie bylo wczesniejszego kontaktu z klientem, w sensie nie kontaktowano sie z nim w ramach poprzedniej kampanii, oraz jest to pierwsza kampania, w ktorej sie z nim kontaktujemy.

### liczba_polaczen_przed_aktualna_kampania

In [ ]:
ctab = pd.crosstab(df.target, df.liczba_polaczen_przed_aktualna_kampania).T
ctab

#### Co ciekawe, im dokonano wiecej polaczen przed aktualna kampania, to wiekszy procent osob decydowal sie na depozyt.

In [ ]:
ctab["%"] = ctab[1] / ctab.sum(axis=1)

In [ ]:
ctab

In [ ]:
sns.countplot(data=df, x=df.liczba_polaczen_przed_aktualna_kampania, hue="target")

In [ ]:
df = df[df.liczba_polaczen_przed_aktualna_kampania <= 3]

### Wspolczynniki rynkowe

In [ ]:
wspolczynniki = (
    "wsk_zmien_zatrudnienia",
    "wsk_cen_konsum",
    "wsk_zauf_konsum",
    "liczba_pracownikow",
)
fig, axs = plt.subplots(len(wspolczynniki), figsize=(12, 24), dpi=200)

for ind, wsp in enumerate(wspolczynniki):
    sns.countplot(
        data=df,
        x=df[wsp],
        ax=axs[ind],
        hue="target",
        dodge=False,
    )

### Spostrzeżenia
- Klienci z mniejszą liczbą pracowników statystycznie częściej zgadzali się na depozyt
-
-

In [ ]:
pd.crosstab([df.liczba_pracownikow, df.wsk_zmien_zatrudnienia], df.target)

#### euribor3m

In [ ]:
sns.kdeplot(
    data=df,
    x="euribor3m",
    hue="target",
);

#### Jakies spostrzezenia
-
-
-

In [ ]:
ctab = (
    pd.crosstab(df.euribor3m, df.target)
    .sort_values(by=[0, 1], ascending=False)
    .head(100)
)
ctab["%"] = ctab[1] / ctab.sum(axis=1) * 100

In [ ]:
ctab

In [ ]:
ctab = (
    pd.crosstab(df.euribor3m, df.target)
    .sort_values(by=[1, 0], ascending=False)
    .head(100)
)
ctab["%"] = ctab[1] / ctab.sum(axis=1) * 100
ctab = ctab.sort_values(by="%", ascending=False)

In [ ]:
ctab

In [ ]:
sns.scatterplot(data=ctab, x=ctab.index, y="%")

#### Im nizsza stopa procentowa, tym wiekszy % osob chce dokonac depozytu.

## Dane kategoryczne

W tej czesci skupimy sie na obserwacjach o wartosciach _unknown_.

In [ ]:
subset = df.select_dtypes(include="object")

In [ ]:
fig, axs = plt.subplots(len(subset.columns) - 3, dpi=200, figsize=(12, 22))

for ind, feature in enumerate(subset.columns):
    if "unknown" in subset[feature].values:
        sns.countplot(data=df, x=df[feature], hue="target", ax=axs[ind], orient="h")
        axs[ind].bar_label(axs[ind].containers[0])
        axs[ind].bar_label(axs[ind].containers[1])
plt.subplots_adjust(hspace=0.8)

### Spostrzezenia
- Rezultaty (Akceptacja bądź nie oferty depozytu) dla rekordów z kolumną o wartości unknown nie odbiegają znacząco od reszty danych
- Dane z polem unknown stanowią małą grupę
- Ze względu na stosunkowo małą grupę i taki sam stosunek akceptacji oferty do odmowy dużym błędem nie byłoby kompletne pominięcię rekordów z polem unknown

### Ma_pozyczke, ma_kredyt, kredyt_mieszkaniowy

In [ ]:
df = df[~(df.ma_pozyczke == "unknown")]
df = df[~(df.ma_kredyt == "unknown")]
df = df[~(df.kredyt_mieszkaniowy == "unknown")]

df.ma_kredyt = (df.ma_kredyt == "no") + 0
df.ma_pozyczke = (df.ma_pozyczke == "no") + 0
df.kredyt_mieszkaniowy = (df.kredyt_mieszkaniowy == "no") + 0

In [ ]:
df.to_csv("data/cleaned.csv")